# 2019 Health Care Ranking Feature Importance Calculator

## Random Forest

### Import Dependencies 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Import the random forest model.
from sklearn.ensemble import RandomForestClassifier 

## This line instantiates the model. 
rf = RandomForestClassifier() 

## Import to decode categorical columns.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

### Read the csv file into a pandas DataFrame

In [9]:
training__counties_data = pd.read_csv('Cleaned_Files/Training_Data_All_Years_All_Counties.csv')
training__counties_data.head()

,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair/Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
0,1000.0,Alabama,0,2020,0.0,9942.794666,22.028703,10.254871,20.927353,35.5,...,16.917263,26.497733,0.703271,1.496909,0.106529,4.443264,65.413428,1.061048,51.633032,40.963183
1,1001.0,Alabama,Autauga,2020,6.0,8128.591190,20.882987,8.619529,18.081557,33.3,...,15.562670,19.343177,0.480207,1.224798,0.111509,2.965774,74.308016,0.820225,51.448715,42.002162
2,1003.0,Alabama,Baldwin,2020,2.0,7354.122530,17.509134,8.345003,17.489033,31.0,...,20.443350,8.783976,0.772399,1.150343,0.066966,4.646779,83.111337,0.543517,51.538377,42.279099
3,1005.0,Alabama,Barbour,2020,45.0,10253.573403,29.591802,11.474559,21.999985,41.7,...,19.420441,48.032635,0.659137,0.454162,0.184880,4.276355,45.641252,1.631683,47.216752,67.789635
4,1007.0,Alabama,Bibb,2020,34.0,11977.539484,19.439724,10.308710,19.114200,37.6,...,16.473214,21.120536,0.437500,0.236607,0.116071,2.625000,74.589286,0.268210,46.781250,68.352607


### 2019 Trainining Data 

In [7]:
Train_Count_2019= training__counties_data[training__counties_data['Year']==2019]
Train_Count_2019.shape

(3132, 62)

### Check in 2019 Training Data number Unique States 

In [18]:
Train_Count_2019['State'].nunique()

51

### Check in 2019 Training Data number Unique Counties 

In [19]:
Train_Count_2019['County/Borough/Parish'].nunique()

1813

### Test County data contains no values under Health_Outcomes_Rank

In [24]:
test__counties_data = pd.read_csv('Cleaned_Files/Test_Data_All_Years_All_Counties.csv')
test__counties_data.head()


,FIPS,State,County/Borough/Parish,Year,Health_Outcomes_Rank,Yrs_Potential_Life_Lost_per_100K,Percent_Fair/Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,...,Percent_Age_65_and_over,Percent_Non-Hispanic_Black_or_African_American,Percent_American_Indian_or_Alaskan_Native,Percent_Asian,Percent_Native_Hawaiian_Other_Pacific_Islander,Percent_Hispanic,Percent_Non-Hispanic_White,Percent_Not_Proficient_in_English,Percent_Female,Percent_Rural
0,2013.0,Alaska,Aleutians East,2020,NR,0.0,16.834521,0.0,16.646811,50.5,...,9.217639,10.412518,19.687055,43.357041,1.194879,14.708393,8.591750,13.084112,32.460882,100.0
1,2060.0,Alaska,Bristol Bay,2020,NR,0.0,15.385269,0.0,18.137644,41.8,...,14.481186,0.798176,32.611174,1.368301,0.342075,7.525656,43.443558,0.000000,46.636260,100.0
2,2230.0,Alaska,Skagway,2020,NR,0.0,11.604994,0.0,15.584385,32.6,...,14.547038,0.696864,5.139373,2.526132,0.348432,5.487805,81.358885,0.402010,47.996516,0.0
3,2282.0,Alaska,Yakutat,2020,NR,0.0,16.437249,0.0,19.879947,31.9,...,19.536424,1.986755,39.569536,6.788079,1.324503,4.635762,32.615894,0.797448,44.536424,100.0
4,8053.0,Colorado,Hinsdale,2020,NR,0.0,12.127612,0.0,12.894843,24.4,...,30.332922,0.616523,1.356350,0.739827,0.000000,6.041924,89.025894,0.000000,48.458693,100.0


### 2019 Test Data 

In [14]:
Test_Count_2019= test__counties_data[test__counties_data['Year']==2019]

### Check in 2019 Test Data number Unique States 

In [15]:
Test_Count_2019['State'].nunique()

15

### Check in 2019 Test Data number Unique Counties 

In [17]:
Test_Count_2019['County/Borough/Parish'].nunique()

59

###  Encode categorical fields

In [30]:
Test_Count_2019= pd.get_dummies(Test_Count_2019)

In [21]:
Train_Count_2019= pd.get_dummies(Train_Count_2019)
Train_Count_2019.shape

(3132, 1926)

### Assign data values to X(_Health_Outcomes_Rank) and y(Health Factors) for model training

### Assign data values for training data only for 2019 

In [22]:
X_train= Train_Count_2019["Health_Outcomes_Rank"]
train= Train_Count_2019.drop(["Health_Outcomes_Rank"], axis=1)

### Assign data values for testing data only for 2019

In [31]:
X_test= Test_Count_2019["Health_Outcomes_Rank_NR"]
test= Test_Count_2019.drop(["Health_Outcomes_Rank_NR"], axis=1)

### Create the classifier model 

In [32]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators= 100)

###  Execute the tuning

In [33]:
model.fit(train, X_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
train_predictions= model.predict(train)

In [37]:
predictions= model.predict(test)

ValueError: Number of features of the model must match the input. Model n_features is 1925 and input n_features is 135 

### Random Forests in sklearn will automatically calculate feature importance

In [ ]:
importances = grid.feature_importances_
importances

### Sort the features by their importance

In [ ]:
 sorted(zip(model.feature_importances_, X.feature_names), reverse=True) 

###  View the results

In [ ]:
print(f"Best parameters: {model.best_params_}")
print(f"Test set score: {model.score(X_test, test)}")